In [1]:
from tokenizer.ml_based_tokenizer import main2
from tokenizer.rule_based_tokenizer import InputType
import pickle

In [2]:
# Let the stemmer take as input the output of the tokenizer (a list of tokens)
file_path = "/Users/lkk/Documents/BOUN CMPE/CMPE 561-Natural Language Processing/Application Project 1/corpora/UD_Turkish-BOUN/tr_boun-ud-test.txt"

token_list = main2(file_path, InputType.FILE_PATH)

In [3]:
print(token_list)

['Çünkü', 'ben', 'de', 'o', 'yaşadığı', 'çevreyi', 'kirleterek', 'bozulmasına', 'neden', 'olan', ',', 'tüm', 'uyarılara', 'kulağını', 'tıkayan', 'soya', 'aitim', '.', 'Kimileri', 'buna', 'kader', 'diyordu', ',', 'kimileri', 'unut', '.', 'Bu', 'da', 'zaman', 'ister', ',', 'emek', 'ister', '.', 'İki', 'veli', 'dokunulsa', 'ağlayacaktı', '.', 'Bu', 'meslek', 'böyledir', '.', 'Filmde', 'kamburunu', 'çıkarmış', 'eski', 'Yugoslav', 'lideri', ',', 'iki', 'kişinin', 'arasında', 'elleri', 'önden', 'bağlı', ',', 'cezaevi', 'avlusunda', 'yürüyor', '.', 'Sabahları', 'hiçbir', 'şey', 'yemeden', 'içmeden', ',', 'paldür', 'küldür', 'çıkıyorum', 'evden', '.', 'Bu', 'albüm', 'hit', 'şarkı', 'çıkarmaz', ',', 'dedim', '.', 'Bu', 'tesisin', ',', "SASA'nın", 'büyümesinde', 'büyük', 'katkısı', 'oldu', '.', 'Çağırdım', 'Genelkurmay', "Başkanı'nı", '.', 'Ama', 'önce', ',', 'uyumadan', 'önce', 'notlarıma', 'bir', 'çeki', 'düzen', 'vermem', 'gerek', '.', 'Saçlarını', 'yıkamış', '.', '"', 'Hayır', ',', 'ben', 'm

In [4]:
tokens = []
for token in token_list:
    if token.isalpha():
        tokens.append(token.lower())

In [5]:
tokens

['çünkü',
 'ben',
 'de',
 'o',
 'yaşadığı',
 'çevreyi',
 'kirleterek',
 'bozulmasına',
 'neden',
 'olan',
 'tüm',
 'uyarılara',
 'kulağını',
 'tıkayan',
 'soya',
 'aitim',
 'kimileri',
 'buna',
 'kader',
 'diyordu',
 'kimileri',
 'unut',
 'bu',
 'da',
 'zaman',
 'ister',
 'emek',
 'ister',
 'i̇ki',
 'veli',
 'dokunulsa',
 'ağlayacaktı',
 'bu',
 'meslek',
 'böyledir',
 'filmde',
 'kamburunu',
 'çıkarmış',
 'eski',
 'yugoslav',
 'lideri',
 'iki',
 'kişinin',
 'arasında',
 'elleri',
 'önden',
 'bağlı',
 'cezaevi',
 'avlusunda',
 'yürüyor',
 'sabahları',
 'hiçbir',
 'şey',
 'yemeden',
 'içmeden',
 'paldür',
 'küldür',
 'çıkıyorum',
 'evden',
 'bu',
 'albüm',
 'hit',
 'şarkı',
 'çıkarmaz',
 'dedim',
 'bu',
 'tesisin',
 'büyümesinde',
 'büyük',
 'katkısı',
 'oldu',
 'çağırdım',
 'genelkurmay',
 'ama',
 'önce',
 'uyumadan',
 'önce',
 'notlarıma',
 'bir',
 'çeki',
 'düzen',
 'vermem',
 'gerek',
 'saçlarını',
 'yıkamış',
 'hayır',
 'ben',
 'masal',
 'istiyorum',
 'helikopterler',
 've',
 'uçakl

In [6]:
len(token_list)

12032

In [7]:

len(tokens)

9598

In [8]:
# Load the suffix and replacement dictionaries from the pickle files
suffix_dict_path = "/Users/lkk/Documents/BOUN CMPE/CMPE 561-Natural Language Processing/Application Project 1/stemmer/suffix_dict.pkl"
replacement_dict_path = "/Users/lkk/Documents/BOUN CMPE/CMPE 561-Natural Language Processing/Application Project 1/stemmer/replacement_dict.pkl"
with open(suffix_dict_path, "rb") as file:
    suffix_dict = pickle.load(file)
with open(replacement_dict_path, "rb") as file:
    replacement_dict = pickle.load(file)

In [9]:
stems = []

In [10]:
def detect_suffix_and_replacement(token, suffix_dict, replacement_dict):
    """
    If exists, detect the suffix part and the corresponding replacement for a token.

    Args:
        token (string): Token to be stemmed
        suffix_dict (dict): The nested hash table that stores suffixes.
        replacement_dict (dict): The dictionary that stores replacements for suffixes.

    Returns:
        suffix (string): Suffix part of the token.
        replacement (string): Part that will replace the removed suffix in the token.
    """
    
    possibleSuffixes = []
    possibleReplacements = []
    
    traversed = ""
    
    current_level = suffix_dict
    
    for char in token[::-1]:
        if char in current_level.keys():
            traversed += char
            current_level = current_level[char]
            if ("END" in current_level) and (current_level["END"] == True):
                possibleSuffix = traversed[::-1]
                possibleSuffixes.append(traversed)
                possibleReplacements.append(replacement_dict[possibleSuffix] if possibleSuffix in replacement_dict else None)
                continue
            else:
                continue
        else:
            break
            
    if len(possibleSuffixes) == 0:
        return None, None
    else:
        for i in range(len(possibleSuffixes)-1, -1, -1):
            suffix = possibleSuffixes[i]
            replacement = possibleReplacements[i]
            
            if replacement is not None:
                possibleStem = token[:-len(suffix)] + replacement
                if len(possibleStem) > 1:
                    return suffix, replacement
                if len(possibleStem) == 1 and possibleStem == "o":
                    return suffix, replacement
                else:
                    continue
            # If replacement is None
            else:
                possibleStem = token[:-len(suffix)]
                if len(possibleStem) > 1:
                    return suffix, None
                if len(possibleStem) == 1 and possibleStem == "o":
                    return suffix, None
                else:
                    continue
    
    return None, None

In [14]:
def detect_suffix_and_replacement2(token, suffix_dict, replacement_dict):
    """
    If exists, detect the suffix part and the corresponding replacement for a token.

    Args:
        token (string): Token to be stemmed
        suffix_dict (dict): The nested hash table that stores suffixes.
        # replacement_dict (dict): The dictionary that stores replacements for suffixes.

    Returns:
        suffix (string): Suffix part of the token.
        # replacement (string): Part that will replace the removed suffix in the token.
    """
    
    possibleSuffixes = []
    # possibleReplacements = []
    
    traversed = ""
    
    current_level = suffix_dict
    
    for char in token[::-1]:
        if char in current_level.keys():
            traversed += char
            current_level = current_level[char]
            if ("END" in current_level) and (current_level["END"] == True):
                possibleSuffix = traversed[::-1]
                possibleSuffixes.append(traversed)
                # possibleReplacements.append(replacement_dict[possibleSuffix] if possibleSuffix in replacement_dict else None)
                continue
            else:
                continue
        else:
            break
            
    if len(possibleSuffixes) == 0:
        return None
    else:
        for i in range(len(possibleSuffixes)-1, -1, -1):
            suffix = possibleSuffixes[i]
            # replacement = possibleReplacements[i]
            
            possibleStem = token[:-len(suffix)]
            if len(possibleStem) > 1:
                return suffix
            if len(possibleStem) == 1 and possibleStem == "o":
                return suffix
            else:
                continue
    
    return None

In [18]:
stems = []

for token in tokens:
    suffix = detect_suffix_and_replacement2(token, suffix_dict, replacement_dict)
    if suffix is not None:
        stems.append(token[:-len(suffix)])
    else:
        stems.append(token)

In [19]:
len(stems)

9598

In [20]:
for i in range(len(stems)):
    print(f"Surface:{tokens[i]}, Stem:{stems[i]}")

Surface:çünkü, Stem:çün
Surface:ben, Stem:be
Surface:de, Stem:de
Surface:o, Stem:o
Surface:yaşadığı, Stem:yaşa
Surface:çevreyi, Stem:çevre
Surface:kirleterek, Stem:kirle
Surface:bozulmasına, Stem:boz
Surface:neden, Stem:ne
Surface:olan, Stem:o
Surface:tüm, Stem:tü
Surface:uyarılara, Stem:uyarı
Surface:kulağını, Stem:kula
Surface:tıkayan, Stem:tıka
Surface:soya, Stem:so
Surface:aitim, Stem:ai
Surface:kimileri, Stem:kim
Surface:buna, Stem:bu
Surface:kader, Stem:ka
Surface:diyordu, Stem:di
Surface:kimileri, Stem:kim
Surface:unut, Stem:unu
Surface:bu, Stem:bu
Surface:da, Stem:da
Surface:zaman, Stem:za
Surface:ister, Stem:ist
Surface:emek, Stem:eme
Surface:ister, Stem:ist
Surface:i̇ki, Stem:i̇
Surface:veli, Stem:ve
Surface:dokunulsa, Stem:dokunul
Surface:ağlayacaktı, Stem:ağlay
Surface:bu, Stem:bu
Surface:meslek, Stem:mesle
Surface:böyledir, Stem:bö
Surface:filmde, Stem:fil
Surface:kamburunu, Stem:kambur
Surface:çıkarmış, Stem:çık
Surface:eski, Stem:es
Surface:yugoslav, Stem:yugoslav
Surfac

In [24]:
b = a[::-1]

In [25]:
a

'acabami'

In [26]:
b

'imabaca'

In [21]:
c = str(b)

In [22]:
c

'<reversed object at 0x17fe9a440>'

In [3]:
from stemmer import main

In [2]:
# Let the stemmer take as input the output of the tokenizer (a list of tokens)
file_path = "/Users/lkk/Documents/BOUN CMPE/CMPE 561-Natural Language Processing/Application Project 1/corpora/UD_Turkish-BOUN/tr_boun-ud-test.txt"

main(file_path, InputType.FILE_PATH)

NameError: name 'StemmerInputType' is not defined